# Your First Binary Simulations with POSYDON 🌠

**Tutorial goal:**
In this tutorial, we will run a small population of 10 binaries locally, and explore how to manipulate the output data from your population.

If you haven't done so yet, export the path POSYDON environment variables.
Set these parameters in your `.bash_profile` or `.zshrc` if you use POSYDON regularly.

In [ ]:
%env PATH_TO_POSYDON=/YOUR/POSYDON/PATH/
%env PATH_TO_POSYDON_DATA=/YOUR/POSYDON_DATA/PATH/


----

## Initialisation File

To run population synthesis with POSYDON, a `population_params.ini` file is required.
This file described how the stellar population is created and what prescriptions and parameters are implemented in specific steps.

POSYDON comes with a default `population_params_default.ini` file found at `PATH_TO_POSYDON/posydon/popsyn` or have a look [here](../../../../../posydon/popsyn/population_params_default.ini).

The parameters in this file can be found [here](../../components-overview/pop_syn/population_params.html), and is split in three parts. You can find more details about their properties by clicking on their links.

1. **[SimulationProperties](../../components-overview/pop_syn/population_params.rst#simulationproperties):**
   - describe the properties and parameters of different steps in the evolution of a binary system.
2. **[BinaryPopulation](../../components-overview/pop_syn/population_params.rst#binarypopulation):** 
   - Parameters of the initial sampling of the binary population, such as initial mass function, period distribution, and metallicity.
   - contains parameters on how the population is run, such as how many binaries are kept in memory.
3. **[SavingOutput](../../components-overview/pop_syn/population_params.rst#saving-output)**
   - Describes what data from the binary and each individual star is written to the output file.

We will copy the default population run parameter file to the current folder.

In [1]:
import os
import shutil
from posydon.config import PATH_TO_POSYDON

path_to_params = os.path.join(PATH_TO_POSYDON, "posydon/popsyn/population_params_default.ini")
shutil.copyfile(path_to_params, './population_params.ini')

'./population_params.ini'


----

## Running a BinaryPopulation in a notebook


<div class="alert alert-warning">

**Reprocessed POSYDON v1 data**

If you're using the reprocessed POSYDON v1 dataset, you will only have solar ($Z_\odot$) metallicity available!
This means you will only be able to run populations with `metallicity = [1]`!
You should be able to follow along with this tutorial and can follow along with the [One metallicity notebook](one_met_pop_syn.ipynb), but the multi-metallicity component is not yet available.

</div>


The copied `population_params.ini` contains the parameters to run 1.000.000 binaries at the eight metallicities currently supported by POSYDON.
This is far too many systems to run into a notebook. So, we will adapt the file to run 10 binaries at $Z_\odot$.

Open the `population_params.ini` file, and go down to the **BinaryPopulation** section. This is where the properties of the simulation are set.
There you should find:

```
metallicity = [2., 1., 0.45, 0.2, 0.1, 0.01, 0.001, 0.0001]
# In units of solar metallicity
...
number_of_binaries = 1000000
# int
```

Let's change these number to the following:
```
metallicity = [1]
...
number_of_binaries = 10
```

If you like to run a small population in a notebook, you can use the `PopulationRunner` ([documentation](../../api_reference/posydon.popsyn.rst#posydon.popsyn.synthetic_population.PopulationRunner)) to do this. If you want to run a specific binary instead, have a look at the [Binary Tutorial](evolve_single_binaries.ipynb) instead. The `PopulationRunner` class takes the `population_params.ini` file and sets-up a multi-metallicity population run for inside a notebook.

It will create `BinaryPopulation`s ([documentation](../../api_reference/posydon.popsyn.rst#posydon.popsyn.binarypopulation.BinaryPopulation)) for each metallicity defined in the `population_params.ini` file. In our case, we can check and see that a single `BinaryPopulation` is created.
We can check if the metallicity and number of binaries are correctly set before starting our simulation. 

In [2]:
from posydon.popsyn.synthetic_population import PopulationRunner
poprun = PopulationRunner('./population_params.ini', verbose=True)

In [3]:
print('Number of binary populations:',len(poprun.binary_populations))
print('Metallicity:', poprun.binary_populations[0].metallicity)
print('Number of binaries:', poprun.binary_populations[0].number_of_binaries)

Number of binary populations: 1
Metallicity: 1.0
Number of binaries: 10



If you changed the parameters in the `population_params.ini` file, you should have the following output:

```
> Number of binary populations: 1
> Metallicity: 1
> Number of binaries: 10
```

The `BinaryPopulation` class does the actual simulation setup and evolution of the binaries at a specific metallicity, but does not take of file cleanup after a succesfull population run. This is why `PopulationRunner` is used in a local environment. In a HPC facility, a setup script is available `posydon-setup-popsyn` that will create the required folders and scripts to run a large population. See the [HPC tutorial](pop_syn.ipynb) for more information.

For this tutorial, we set `verbose=True`, which shows you the progress of the population run.
This overwrites the population verbose set inside the `population_params.ini` file.
Now we are ready to evolve the binary population. This should take about 30 seconds depending on your machine.

In [4]:
poprun.evolve()

 10%|█         | 1/10 [00:03<00:29,  3.23s/it]/Users/max/Documents/POSYDON/posydon/binary_evol/DT/step_detached.py:1690: RuntimeWarning: invalid value encountered in log10
  current = np.log10(
 70%|███████   | 7/10 [00:08<00:01,  1.51it/s]/Users/max/Documents/POSYDON/posydon/binary_evol/DT/step_detached.py:1690: RuntimeWarning: invalid value encountered in log10
  current = np.log10(
 80%|████████  | 8/10 [00:08<00:01,  1.61it/s]/Users/max/Documents/POSYDON/posydon/binary_evol/DT/step_detached.py:1690: RuntimeWarning: invalid value encountered in log10
  current = np.log10(
100%|██████████| 10/10 [00:12<00:00,  1.23s/it]


Merging 1 files...
Files merged!
Removing files in /Users/max/Documents/POSYDON/docs/_source/tutorials-examples/population-synthesis/1e+00_Zsun_batches...



-----

## Inspecting the population: Population class

When you ran the population, you might have seen that a temporary folder with the name `1e+00_Zsun_batches` was created while the binaries were being evolved. This is a temporary folder in which populations are temporarly saved.
After the binary evolution has finished, the binaries in the folder are moved to a single file named `1e+00_Zsun_popululation.h5`. This is done automatically when you run a population using the `PopulationRunner` class.
When you run multiple metallicity a file will be created for each metallicity.

The created file contains 3 main components:

1. **history:** the evolution of an individual binary in a pandas DataFrame
2. **oneline:** a single line to describe the initial and final conditions and some one-of parameters, such as the metallicity.
3. **mass_per_metallicity:** some metadata on the population, such as the total simulated mass, the actual underlying mass of the population, and the number of binaries in the file.

The `Population` class provides an interface to these components in the file, such that you're able to share the populations runs and can work with large populations that do not fit in memory. We will now explore the population file using the `Population` class. You can find a more extensive description [here](../../components-overview/pop_syn/synthetic_population.rst) or look at the class [documentation](../../api_reference/posydon.popsyn.rst#posydon.popsyn.synthetic_population.Population).


<div class="alert alert-warning">

**Older Population Files**

If you're using older population files from before the Population class rework, you can make them compatible with the `Population` class by calling `Population(pop_file, metallicity,ini_file)`, where the `metallicity` is in solar units. You will only need to do this once; afterwards you can initialise the class like normal.

</div>

In [5]:
from posydon.popsyn.synthetic_population import Population
pop = Population('1e+00_Zsun_population.h5')


Let's start with the `pop.mass_per_metallicity`. 
It contains some basic information about the population you've just created.

1. The index (`metallicity`) is the metallicity of your population in solar units.
2. `simulated mass` the total ZAMS mass that has been evolved in the population.
3. `simulated_mass_single` the total ZAMS mass from initially single stars.
4. `simulated_mass_binaries` the total ZAMS mass from initially binaries.
5. `number_of_systems` shows the number of systems in the file.


In [6]:
pop.mass_per_metallicity

,simulated_mass,simulated_mass_single,simulated_mass_binaries,number_of_systems
metallicity,,,,
1.0,240.819419,0.0,240.819419,10



There are some additional metadata properties available, such as:

- `metallicities` the metallicity in absolute metallicity
- `solar_metallicities` the metallicities in the file in solar metallicity
- `number_of_systems` the total number of systems in the Population file
- `indices` the indices of the binaries in the file
- `columns` the columns available in the `history` and `oneline` dataframes
- `ini_params` the parameters from the `ini` file that describe the initial sampling of your population.


In [7]:
# you can also access the total number of systems in the file with
print(pop.number_of_systems)

10


## Population.history

`pop.history` contains the evolutionary histories of each binary, as it was evolved by POSYDON. You can find more information about this [here](../../components-overview/pop_syn/synthetic_population.rst#history) (or look at the [documentation](../../api_reference/posydon.popsyn.rst#posydon.popsyn.synthetic_population.History) of the class).


It allows you to load specific information or binaries into memory without having to load them all at once.

<div class="alert alert-info">

**Note**

Calling `pop.history` will load all binaries and all their columns into memory. This can take a while and can even cause the notebook to crash.

</div>

You can access individual or selections of the population using several methods:

```
1. pop.history[5]
2. pop.history[[0,4]]
3. pop.history['time]
4. pop.history.select()
```

The `select` function is the most powerfull way to access the binaries, because it allows you to perform selections based on the specific columns available in the history dataframe.
For example, below we can select on `state == 'RLO1'`, which gives us all the rows with RLO1 occuring.

The available identifiers are limited to string columns (`state`, `event`, `step_names`, `S1_state`, `S2_state`), index, and columns names.

<div class="alert alert-warning">

**Not all columns are available**

It's not currently possible to select on all columns in the population file. **Only string columns, the indices and columns names are available!** 

</div>


In [8]:
# select only binary_index 5
pop.history[5]

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
5,detached,ZAMS,0.000000e+00,16.100059,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,11.036583,...,NaN,NaN,NaN,NaN,0.715500,2.703000e-01,NaN,NaN,NaN,NaN
5,detached,CC1,2.426487e+07,13.510368,0.0,-99.0,step_HMS_HMS,0.010408,stripped_He_Central_C_depletion,3.414923,...,0.000000,0.0,0.000000,0.000000,0.573707,4.123814e-01,0.710422,0.275292,0.805838,15.584856
5,disrupted,NaN,2.426487e+07,NaN,NaN,NaN,step_SN,0.000151,NS,1.150693,...,0.000000,0.0,0.000000,0.000000,0.573707,4.123814e-01,0.710422,0.275292,0.805838,15.584856
5,disrupted,CC2,6.905538e+07,NaN,NaN,NaN,step_disrupted,0.321384,NS,1.150693,...,1.370221,NaN,1.172709,0.029706,0.000000,1.259616e-14,0.670076,0.315632,0.017749,13.344821
5,disrupted,NaN,6.905538e+07,NaN,NaN,NaN,step_SN,0.000049,NS,1.150693,...,1.172709,NaN,1.172709,NaN,0.000000,1.259616e-14,NaN,NaN,NaN,0.000000
5,disrupted,END,6.905538e+07,NaN,NaN,NaN,step_end,0.000015,NS,1.150693,...,1.172709,NaN,1.172709,NaN,0.000000,1.259616e-14,NaN,NaN,NaN,0.000000


In [9]:
# select binary 0 and 4
pop.history[[0,4]]

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
0,detached,ZAMS,0.000000e+00,7.477197,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,16.045066,...,NaN,NaN,NaN,NaN,0.715500,2.703000e-01,NaN,NaN,NaN,NaN
0,detached,CC1,1.383148e+07,3.953997,0.0,-99.000000,step_HMS_HMS,0.631011,stripped_He_Central_C_depletion,4.719060,...,0.000000,0.0,0.000000,0.000000,0.580364,4.057340e-01,0.567593,0.418279,0.956708,19.165133
0,disrupted,NaN,1.383148e+07,NaN,NaN,NaN,step_SN,0.000311,NS,1.388635,...,0.000000,0.0,0.000000,0.000000,0.580364,4.057340e-01,0.567593,0.418279,0.956708,19.165133
0,disrupted,CC2,3.664020e+07,NaN,NaN,NaN,step_disrupted,2.537633,NS,1.388635,...,3.328431,NaN,2.112493,0.052222,0.000000,8.141926e-14,0.687992,0.297731,0.007116,5.819024
0,disrupted,NaN,3.664020e+07,NaN,NaN,NaN,step_SN,0.000506,NS,1.388635,...,NaN,NaN,2.112493,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
0,disrupted,END,3.664020e+07,NaN,NaN,NaN,step_end,0.000020,NS,1.388635,...,NaN,NaN,2.112493,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
4,detached,ZAMS,0.000000e+00,4.362458,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,33.117780,...,NaN,NaN,NaN,NaN,0.715500,2.703000e-01,NaN,NaN,NaN,NaN
4,contact,oCE1,4.950971e+06,1.508327,0.0,-0.507244,step_HMS_HMS,0.010695,H-rich_Core_H_burning,21.640368,...,0.000000,0.0,0.000000,0.000000,0.615545,3.705509e-01,0.631692,0.355040,0.989788,14.488990
4,merged,oMerging1,4.950971e+06,1.508327,0.0,-0.507244,step_CE,0.000027,H-rich_Core_H_burning,21.640368,...,0.000000,0.0,0.000000,0.000000,0.615545,3.705509e-01,0.631692,0.355040,0.989788,14.488990


In [10]:
pop.history['time'].head()

,time
binary_index,
0,0.000000e+00
0,1.383148e+07
0,1.383148e+07
0,3.664020e+07
0,3.664020e+07



You can also check what columns are available in the history file.
This is possible in two ways:

In [11]:
pop.history.columns

['state',
 'event',
 'time',
 'orbital_period',
 'eccentricity',
 'lg_mtransfer_rate',
 'step_names',
 'step_times',
 'S1_state',
 'S1_mass',
 'S1_log_R',
 'S1_log_L',
 'S1_lg_mdot',
 'S1_he_core_mass',
 'S1_he_core_radius',
 'S1_co_core_mass',
 'S1_co_core_radius',
 'S1_center_h1',
 'S1_center_he4',
 'S1_surface_h1',
 'S1_surface_he4',
 'S1_surf_avg_omega_div_omega_crit',
 'S1_spin',
 'S2_state',
 'S2_mass',
 'S2_log_R',
 'S2_log_L',
 'S2_lg_mdot',
 'S2_he_core_mass',
 'S2_he_core_radius',
 'S2_co_core_mass',
 'S2_co_core_radius',
 'S2_center_h1',
 'S2_center_he4',
 'S2_surface_h1',
 'S2_surface_he4',
 'S2_surf_avg_omega_div_omega_crit',
 'S2_spin']

In [12]:
pop.columns['history']

['state',
 'event',
 'time',
 'orbital_period',
 'eccentricity',
 'lg_mtransfer_rate',
 'step_names',
 'step_times',
 'S1_state',
 'S1_mass',
 'S1_log_R',
 'S1_log_L',
 'S1_lg_mdot',
 'S1_he_core_mass',
 'S1_he_core_radius',
 'S1_co_core_mass',
 'S1_co_core_radius',
 'S1_center_h1',
 'S1_center_he4',
 'S1_surface_h1',
 'S1_surface_he4',
 'S1_surf_avg_omega_div_omega_crit',
 'S1_spin',
 'S2_state',
 'S2_mass',
 'S2_log_R',
 'S2_log_L',
 'S2_lg_mdot',
 'S2_he_core_mass',
 'S2_he_core_radius',
 'S2_co_core_mass',
 'S2_co_core_radius',
 'S2_center_h1',
 'S2_center_he4',
 'S2_surface_h1',
 'S2_surface_he4',
 'S2_surf_avg_omega_div_omega_crit',
 'S2_spin']

In [13]:
# using the select function
pop.history.select(where='index == 9')

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
9,detached,ZAMS,0.000000e+00,5.511395,0.000000,NaN,initial_cond,0.000000,H-rich_Core_H_burning,29.609117,...,NaN,NaN,NaN,NaN,0.71550,2.703000e-01,NaN,NaN,NaN,NaN
9,detached,CC1,7.239310e+06,8.080874,0.000000,-99.0,step_HMS_HMS,0.579610,stripped_He_Central_C_depletion,9.149855,...,7.823369,0.446786,6.877524,0.168205,0.03902,9.969697e-09,0.164474,0.739482,0.053132,0.652143
9,detached,NaN,7.239310e+06,28.941227,0.570005,NaN,step_SN,0.000383,BH,8.649855,...,7.823369,0.446786,6.877524,0.168205,0.03902,9.969697e-09,0.164474,0.739482,0.053132,0.652143
9,detached,redirect_from_CO_HeMS,7.239310e+06,28.941227,0.570005,NaN,step_CO_HeMS,0.000025,BH,8.649855,...,7.823369,0.446786,6.877524,0.168205,0.03902,9.969697e-09,0.164474,0.739482,0.053132,0.652143
9,detached,CC2,7.245894e+06,22.066910,0.354742,NaN,step_detached,2.493930,BH,8.649855,...,10.463428,NaN,8.457174,0.113635,0.00000,3.490631e-13,0.300510,0.685631,0.160424,0.023880
9,detached,NaN,7.245894e+06,19.170613,0.401599,NaN,step_SN,0.031193,BH,8.649855,...,NaN,NaN,8.457174,NaN,NaN,NaN,NaN,NaN,NaN,0.005501
9,detached,maxtime,1.380000e+10,19.098267,0.400646,NaN,step_dco,0.002207,BH,8.649855,...,NaN,NaN,8.457174,NaN,NaN,NaN,NaN,NaN,NaN,0.005501
9,detached,END,1.380000e+10,19.098267,0.400646,NaN,step_end,0.000016,BH,8.649855,...,NaN,NaN,8.457174,NaN,NaN,NaN,NaN,NaN,NaN,0.005501


In [14]:
# selecting all RLO1 states and only time and state columns
pop.history.select(where='state == RLO2', columns=['time', 'state'])

,time,state
binary_index,,


In [15]:
# selecting rows 10 to 16
pop.history.select(start=10, stop=16)

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
1,disrupted,NaN,2.671979e+08,NaN,NaN,NaN,step_SN,0.000068,NS,1.150693,...,0.773876,NaN,0.773876,NaN,0.000000,1.003437e-28,NaN,NaN,NaN,0.000000
1,disrupted,END,2.671979e+08,NaN,NaN,NaN,step_end,0.000018,NS,1.150693,...,0.773876,NaN,0.773876,NaN,0.000000,1.003437e-28,NaN,NaN,NaN,0.000000
2,detached,ZAMS,0.000000e+00,4.957037,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,7.450705,...,NaN,NaN,NaN,NaN,0.715500,2.703000e-01,NaN,NaN,NaN,NaN
2,contact,oCE1,4.732243e+07,3.582680,0.0,-3.430261,step_HMS_HMS,0.065059,H-rich_Core_H_burning,7.095651,...,0.000000,0.0,0.000000,0.0,0.699916,2.855205e-01,0.715499,0.270271,0.748479,2.084443
2,merged,oMerging1,4.732243e+07,3.582680,0.0,-3.430261,step_CE,0.000021,H-rich_Core_H_burning,7.095651,...,0.000000,0.0,0.000000,0.0,0.699916,2.855205e-01,0.715499,0.270271,0.748479,2.084443
2,merged,CC1,5.292376e+07,NaN,NaN,NaN,step_merged,2.538206,H-rich_Central_C_depletion,7.390554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


You might have notices while using the above functions that not all the binaries will have the same length in the history.
You can access these with `pop.history_lengths` or `pop.history.lengths`. They provide the same information.

In [16]:
pop.history_lengths

,length
index,
0,6
1,6
2,6
3,7
4,6
5,6
6,6
7,6
8,6


In [17]:
pop.history.lengths

,length
index,
0,6
1,6
2,6
3,7
4,6
5,6
6,6
7,6
8,6



-----

## Population.oneline

`Population.oneline` provides a similar interface to accessing the DataFrame in the population file as `Population.history`, with similar functionality being available.

The `oneline` DataFrame contains, as the name suggests, a single line per binary. It contains initial and final conditions and some additional varibales, such as the `SN_type`.
You can find more information about this [here](../../components-overview/pop_syn/synthetic_population.rst#oneline) or look at the [api reference](../../api_reference/posydon.popsyn.rst#posydon.popsyn.synthetic_population.Oneline) of the class.


The `select` function only has access to:

- `index`
- column names 
- string columns: `state_i`, `state_f`, `event_i`, `event_f`, `step_names_i`, `step_names_f`, `S1_state_i`, `S1_state_f`, `S2_state_i`, `S2_state_f`, `S1_SN_type`, `S2_SN_type`, `interp_class_HMS_HMS`, `interp_class_CO_HeMS`, `interp_class_CO_HMS_RLO`, `interp_class_CO_HeMS_RLO`, `mt_history_HMS_HMS`, `mt_history_CO_HeMS`, `mt_history_CO_HMS_RLO`, `mt_history_CO_HeMS_RLO`

In [18]:
pop.oneline[5]

,state_i,event_i,time_i,orbital_period_i,eccentricity_i,lg_mtransfer_rate_i,step_names_i,step_times_i,state_f,event_f,...,interp_class_CO_HMS_RLO,interp_class_CO_HeMS,interp_class_CO_HeMS_RLO,mt_history_HMS_HMS,mt_history_CO_HMS_RLO,mt_history_CO_HeMS,mt_history_CO_HeMS_RLO,FAILED,WARNING,metallicity
binary_index,,,,,,,,,,,,,,,,,,,,,
5,detached,ZAMS,0.0,16.100059,0.0,NaN,initial_cond,0.0,disrupted,END,...,NaN,NaN,NaN,Stable RLOF during postMS,NaN,NaN,NaN,0,1,1.0


In [19]:
pop.oneline[[0,4]]

,state_i,event_i,time_i,orbital_period_i,eccentricity_i,lg_mtransfer_rate_i,step_names_i,step_times_i,state_f,event_f,...,interp_class_CO_HMS_RLO,interp_class_CO_HeMS,interp_class_CO_HeMS_RLO,mt_history_HMS_HMS,mt_history_CO_HMS_RLO,mt_history_CO_HeMS,mt_history_CO_HeMS_RLO,FAILED,WARNING,metallicity
binary_index,,,,,,,,,,,,,,,,,,,,,
0,detached,ZAMS,0.0,7.477197,0.0,NaN,initial_cond,0.0,disrupted,END,...,NaN,NaN,NaN,Stable RLOF during postMS,NaN,NaN,NaN,0,1,1.0
4,detached,ZAMS,0.0,4.362458,0.0,NaN,initial_cond,0.0,merged,END,...,NaN,NaN,NaN,Unstable RLOF during MS,NaN,NaN,NaN,0,1,1.0


In [20]:
pop.oneline.select(where='index == 9')

,state_i,event_i,time_i,orbital_period_i,eccentricity_i,lg_mtransfer_rate_i,step_names_i,step_times_i,state_f,event_f,...,interp_class_CO_HMS_RLO,interp_class_CO_HeMS,interp_class_CO_HeMS_RLO,mt_history_HMS_HMS,mt_history_CO_HMS_RLO,mt_history_CO_HeMS,mt_history_CO_HeMS_RLO,FAILED,WARNING,metallicity
binary_index,,,,,,,,,,,,,,,,,,,,,
9,detached,ZAMS,0.0,5.511395,0.0,NaN,initial_cond,0.0,detached,END,...,NaN,NaN,NaN,Stable contact phase,NaN,NaN,NaN,0,1,1.0


In [21]:
pop.oneline.select(where='index == [0,9]')

,state_i,event_i,time_i,orbital_period_i,eccentricity_i,lg_mtransfer_rate_i,step_names_i,step_times_i,state_f,event_f,...,interp_class_CO_HMS_RLO,interp_class_CO_HeMS,interp_class_CO_HeMS_RLO,mt_history_HMS_HMS,mt_history_CO_HMS_RLO,mt_history_CO_HeMS,mt_history_CO_HeMS_RLO,FAILED,WARNING,metallicity
binary_index,,,,,,,,,,,,,,,,,,,,,
0,detached,ZAMS,0.0,7.477197,0.0,NaN,initial_cond,0.0,disrupted,END,...,NaN,NaN,NaN,Stable RLOF during postMS,NaN,NaN,NaN,0,1,1.0
9,detached,ZAMS,0.0,5.511395,0.0,NaN,initial_cond,0.0,detached,END,...,NaN,NaN,NaN,Stable contact phase,NaN,NaN,NaN,0,1,1.0


In [22]:
pop.oneline.columns
# or 
# pop.columns['oneline']

['state_i',
 'event_i',
 'time_i',
 'orbital_period_i',
 'eccentricity_i',
 'lg_mtransfer_rate_i',
 'step_names_i',
 'step_times_i',
 'state_f',
 'event_f',
 'time_f',
 'orbital_period_f',
 'eccentricity_f',
 'lg_mtransfer_rate_f',
 'step_names_f',
 'step_times_f',
 'S1_state_i',
 'S1_mass_i',
 'S1_log_R_i',
 'S1_log_L_i',
 'S1_lg_mdot_i',
 'S1_he_core_mass_i',
 'S1_he_core_radius_i',
 'S1_co_core_mass_i',
 'S1_co_core_radius_i',
 'S1_center_h1_i',
 'S1_center_he4_i',
 'S1_surface_h1_i',
 'S1_surface_he4_i',
 'S1_surf_avg_omega_div_omega_crit_i',
 'S1_spin_i',
 'S1_state_f',
 'S1_mass_f',
 'S1_log_R_f',
 'S1_log_L_f',
 'S1_lg_mdot_f',
 'S1_he_core_mass_f',
 'S1_he_core_radius_f',
 'S1_co_core_mass_f',
 'S1_co_core_radius_f',
 'S1_center_h1_f',
 'S1_center_he4_f',
 'S1_surface_h1_f',
 'S1_surface_he4_f',
 'S1_surf_avg_omega_div_omega_crit_f',
 'S1_spin_f',
 'S1_natal_kick_array_0',
 'S1_natal_kick_array_1',
 'S1_natal_kick_array_2',
 'S1_natal_kick_array_3',
 'S1_SN_type',
 'S1_f_fb',
 


-----

## Population.formation_channels

While you can see the all the main evolutionary steps in the evolution of a binary, it is useful to have a summary overview of a binary's evolutionary pathway, also known as formation channel

You might be interested in figuring out what sort of formation pathways/channels a binary has followed through its evolution.

This is not a standard output of the population synthesis, but you can include it into the population file by calculating it. 
If you would like more detail on the initial mass transfer, you can set `mt_history=True`.

This will write the formation channels to the Population file, which can be accessed by `Population.formation_channels`.


In [23]:
pop.calculate_formation_channels(mt_history=True)

In [24]:
# the formation channels are loaded in with pop.formation_channels
pop.formation_channels

,channel_debug,channel
0,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
1,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
2,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
3,ZAMS_redirect_from_ZAMS_CC1_CC2_END,ZAMS_CC1_CC2_END
4,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
5,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
6,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
7,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
8,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
9,ZAMS_oRLO1_CC1_redirect_from_CO_HeMS_CC2_maxti...,ZAMS_oRLO1-contact_CC1_CC2_maxtime_END


Next time you open this population file, the `formation_channels` will be available without having to be recalculated.

In [25]:
pop = Population('1e+00_Zsun_population.h5')
pop.formation_channels

,channel_debug,channel
0,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
1,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
2,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
3,ZAMS_redirect_from_ZAMS_CC1_CC2_END,ZAMS_CC1_CC2_END
4,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
5,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
6,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
7,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
8,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
9,ZAMS_oRLO1_CC1_redirect_from_CO_HeMS_CC2_maxti...,ZAMS_oRLO1-contact_CC1_CC2_maxtime_END


## Selecting a sub-population

You might just want a small sub-selection of the full population, especially if you're working with large population and multi-metallicity runs.

The `Population.export_selection()` function will export just the indices of the binaries you're interested in into a new file.
The simulated mass will remain the same, since they are dependent on the population run.

If we select just 2 binaries and export them, we create a new population of just the binaries you're interested in.
In the [BBH analysis](bbh_analysis.ipynb) and [GRB analysis](lgrb_pop_syn.ipynb) tutorials, we show how to perform a selection with multiple criteria and across metallicities.



In [27]:
indices = [0,9]
pop.export_selection(indices, 'selected.h5')

In [28]:
selected = Population('selected.h5')
selected.mass_per_metallicity

,simulated_mass,simulated_mass_single,simulated_mass_binaries,number_of_systems
metallicity,,,,
1.0,240.819419,0.0,240.819419,2


If you would like to know the simulated mass of just your population, you can calulate this using the online ZAMS values.

In [29]:
import numpy as np

print('selected simulated mass: ', np.sum(selected.oneline[['S1_mass_i', 'S2_mass_i']].to_numpy()))

selected simulated mass:  83.33858151818127


By default this export-selection will not overwrite nor append if the output file is already present.
You have to explicitly state what you would like to append to or overwrite the population file.


With the `append=True` you are able to combine multiple stellar populations into a single file. 
This is especially useful when creating multi-metallicity populations.

In [30]:
# this will overwrite the existing file
pop.export_selection(indices, 'selected.h5', overwrite=True)

selected = Population('selected.h5')
selected.mass_per_metallicity


,simulated_mass,simulated_mass_single,simulated_mass_binaries,number_of_systems
metallicity,,,,
1.0,240.819419,0.0,240.819419,2


In [31]:
# This will add to the file and add the extra simulated mass
pop.export_selection(indices, 'selected.h5', append=True)
selected = Population('selected.h5')
selected.mass_per_metallicity

,simulated_mass,simulated_mass_single,simulated_mass_binaries,number_of_systems
metallicity,,,,
1.0,481.638838,0.0,481.638838,4


<br/><br/>

Feel free to explore the small binary population you've just created!

If you want to learn more about population synthesis and how to perform more complex selection and population, continue with [Large scale population on a HPC setup](pop_syn.ipynb) or with [BBH analysis](bbh_analysis.ipynb).


----

### Local MPI runs

To speed up population synthesis runs, you can run on a computing cluster, as described in [HPC Facilities](pop_syn.ipynb), or you can distribute the population synthesis across multiple cores on your local machine using MPI.

To enable local MPI runs, go into the `population_params.ini`  and change `use_MPI` to `True`.

It's important to note that you cannot run have this option enabled for cluster runs!

We create a binary population simulation script to run the population:

In [ ]:
%%writefile script.py
from posydon.popsyn.synthetic_population import PopulationRunner

if __name__ == "__main__":
    synth_pop = PopulationRunner("./population_params.ini")
    synth_pop.evolve()

This script can be initiated using a local where `NR_processors` is the number of processors you would like to us.
`mpi4py` needs to be installed for this. Please see the [Installation guide](../../getting-started/installation-guide.rst) for more info.

In [ ]:
mpiexec -n ${NR_processors} python script.py

This will create a folder for each metallicity in the population and store output of the parallel runs in it.

You will have to concatenate these runs manually into a single population file per metallicity, which can be achieved using the following code:

In [ ]:
from posydon.popsyn.synthetic_population import PopulationRunner

synth_pop = PopulationRunner("./population_params.ini")
for pop in synth_pop.binary_populations:
    synth_pop.merge_parallel_runs(pop)